In [3]:
import numpy as np
import scipy
import scipy.io

In [4]:
qm7_path = "/home/jonnyjack/workspace/FPTAI/research-test/son-hy/qm7.mat"
dataset = scipy.io.loadmat(qm7_path)
dataset.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'R', 'Z', 'T', 'P'])

In [5]:
print("Couloumb matrices:", dataset['X'].shape)
print("Labels (Atomization energies):", dataset['T'].shape)
print("Splits for cross-validation:", dataset['P'].shape)
print("Cartesian cooridnate:", dataset['R'].shape)
print("Atomic charge:", dataset['Z'].shape)

Couloumb matrices: (7165, 23, 23)
Labels (Atomization energies): (1, 7165)
Splits for cross-validation: (5, 1433)
Cartesian cooridnate: (7165, 23, 3)
Atomic charge: (7165, 23)


In [6]:
dataset["P"][0].max()

7161

In [10]:
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

svr = SVR(kernel='rbf', gamma=0.1)
kr = KernelRidge(kernel='rbf', gamma=0.1)
ridge = Ridge(alpha=1.0)
scaler = StandardScaler()

In [12]:
no_splits = len(dataset["P"])
for split in range(no_splits):
    X_test = dataset["X"][dataset["P"][split]]
    y_test = dataset["T"][0, dataset["P"][split]]
    X_train = dataset["X"][dataset["P"][list(range(0, split)) + list(range(split + 1, no_splits))].flatten()]
    y_train = dataset["T"][0, dataset["P"][list(range(0, split)) + list(range(split + 1, no_splits))].flatten()]
    num_train_samples = X_train.shape[0]
    num_test_samples = X_test.shape[0]
    # X_train_scaled = scaler.fit_transform(X_train.reshape(num_train_samples, -1))
    # X_test_scaled = scaler.transform(X_test.reshape(num_test_samples, -1))
    kr.fit(X_train.reshape(num_train_samples, -1), y_train)
    y_pred = kr.predict(X_test.reshape(num_test_samples, -1))
    # svr.fit(X_train_scaled, y_train)
    # y_pred = svr.predict(X_test_scaled)
    print("Split %d" % split)
    print("MAE: %5.2f kcal/mol" % mean_absolute_error(y_test, y_pred))
    print("RMSE: %5.2f kcal/mol" % mean_squared_error(y_test, y_pred)**.5)


Split 0
MAE: 1537.64 kcal/mol
RMSE: 1553.94 kcal/mol
Split 1
MAE: 1537.55 kcal/mol
RMSE: 1553.85 kcal/mol
Split 2
MAE: 1537.43 kcal/mol
RMSE: 1553.64 kcal/mol
Split 3
MAE: 1537.76 kcal/mol
RMSE: 1553.95 kcal/mol
Split 4
MAE: 1537.79 kcal/mol
RMSE: 1554.00 kcal/mol


In [17]:
kr.n_features_in_

529